In [ ]:
import json
import requests
from datetime import datetime
import pytz

currency_pair = "btcusd"
# ohlc_url = f"https://www.bitstamp.net/api/v2/ticker/{currency_pair}/"
ohlc_url = f'https://www.bitstamp.net/api/v2/ohlc/{currency_pair}/?step=3600&limit=1'
hour_url = "http://127.0.0.1:8000/scrap/api/bitstamphour/add"
day_url = "http://127.0.0.1:8000/scrap/api/bitstamp/add"
error_log_url = "http://127.0.0.1:8000/scrap/api/error_log/add/"

def call_bitsamp_api(ohlc_url,api_endpoint,error_log_url):
    ohlc_data_dict = {}

    try:
        data = requests.get(ohlc_url)

        bitstamp_data = data.json()['data']

        ohlc_data = bitstamp_data['ohlc'][0]
        ohlc_data['unix'] = ohlc_data.pop('timestamp')
        ohlc_data['volume_btc'] = ohlc_data.pop('volume')
        # Convert the Unix timestamp to UTC time
        ohlc_data['date'] = datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')
        
        # PROD
        # ohlc_data['symbol'] = bitstamp_data['pair']

        # TEST
        ohlc_data['symbol'] = 'TEST'

        # Convert the timestamp to datetime and localize it to UTC
        dt = datetime.fromtimestamp(int(ohlc_data['unix']), tz=pytz.UTC)
        # Convert the datetime to Kuala Lumpur time
        kl_time = dt.astimezone(pytz.timezone('Asia/Kuala_Lumpur'))
        ohlc_data['local_date'] = kl_time.strftime('%Y-%m-%dT%H:%M:%S')

        # Convert 'volume' and 'close' to float before multiplication
        ohlc_data['volume_usd'] = float(ohlc_data['volume_btc']) * float(ohlc_data['close'])
        
        ohlc_json = json.dumps(ohlc_data)

        # Convert the string to a dictionary
        ohlc_data_dict = json.loads(ohlc_json)

    except Exception as e:
        print(e)
        error = {
            "error_message": e.message if hasattr(e, 'message') else str(e),
            "error_source": 'when calling bitstamp official api',
            "error_type": 'bitstamp api error',
            "error_content": data.to_dict() if hasattr(data, 'to_dict') else str(data)
        }
        requests.post(error_log_url, data=error)
        
    try:
        response = requests.post(api_endpoint, json=ohlc_data_dict)
        if response.status_code == 200:
            print('Data successfully added to the database')
        else:

            error_data = {
                "error_source": 'Bitstamp API',
                "error_type": response.text,
                "error_content": api_endpoint,
                "error_message": "Calling bitstamp store api"
            }
            requests.post(error_log_url, json=error_data)
    except requests.exceptions.RequestException as e:
        error_data = {
        "error_source": 'Bitstamp API',
        "error_type": response.text,
        "error_content": api_endpoint,
        "error_message": e.message if hasattr(e, 'message') else str(e),
        }
        requests.post(error_log_url, json=error_data)
        print(e)


Dev

In [1]:
import json
import requests
import datetime
import pytz
# import pandas as pd
# import datetime
# from datetime import date

In [9]:
currency_pair = "btcusd"
# ohlc_url = f"https://www.bitstamp.net/api/v2/ticker/{currency_pair}/"
ohlc_url = f'https://www.bitstamp.net/api/v2/ohlc/{currency_pair}/?step=3600&limit=1'
hour_url = "http://127.0.0.1:8000/scrap/api/bitstamp/add"
error_log_url = "http://127.0.0.1:8000/scrap/api/error_log/add/"

In [26]:
try:
    data = requests.get(ohlc_url)
    bitstamp_data = data.json()['data']
    ohlc_data = bitstamp_data['ohlc'][0]
    ohlc_data['unix'] = ohlc_data.pop('timestamp')
    ohlc_data['volume_btc'] = ohlc_data.pop('volume')
    # Convert the Unix timestamp to UTC time
    ohlc_data['date'] = datetime.datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')
    # PROD
    # ohlc_data['symbol'] = bitstamp_data['pair']

    # TEST
    ohlc_data['symbol'] = 'TEST'

    # Convert the timestamp to datetime and localize it to UTC
    dt = datetime.datetime.fromtimestamp(int(ohlc_data['unix']), tz=pytz.UTC)
    # Convert the datetime to Kuala Lumpur time
    kl_time = dt.astimezone(pytz.timezone('Asia/Kuala_Lumpur'))
    ohlc_data['local_date'] = kl_time.strftime('%Y-%m-%dT%H:%M:%S')

    # Convert 'volume' and 'close' to float before multiplication
    ohlc_data['volume_usd'] = float(ohlc_data['volume_btc']) * float(ohlc_data['close'])
    ohlc_json = json.dumps(ohlc_data)
    display(ohlc_json)
    # Convert the string to a dictionary
    ohlc_data_dict = json.loads(ohlc_json)

except Exception as e:
    print(e)
    error = {
        "error_message": e.message if hasattr(e, 'message') else str(e),
        "error_source": 'when calling bitstamp official api',
        "error_type": 'bitstamp api error',
        "error_content": data.to_dict() if hasattr(data, 'to_dict') else str(data)
    }
    requests.post(error_log_url, data=error)

C:\Users\Zhong_ming_Tan\AppData\Local\Temp\ipykernel_42892\1201533626.py:8: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  ohlc_data['date'] = datetime.datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')


'{"close": "51503", "high": "51763", "low": "51441", "open": "51735", "unix": "1708873200", "volume_btc": "42.98860061", "date": "2024-02-25T15:00:00", "symbol": "TEST", "local_date": "2024-02-25T23:00:00", "volume_usd": 2214041.89721683}'

In [86]:
data = requests.get(ohlc_url)
bitstamp_data = data.json()['data']
ohlc_data = bitstamp_data['ohlc'][0]
display(ohlc_data)
display(bitstamp_data['pair'])

{'close': '51759',
 'high': '51763',
 'low': '51706',
 'open': '51735',
 'timestamp': '1708873200',
 'volume': '0.79554625'}

'BTC/USD'

In [87]:
ohlc_data['unix'] = ohlc_data.pop('timestamp')
ohlc_data['volume_btc'] = ohlc_data.pop('volume')

In [18]:
# Convert the Unix timestamp to UTC time
ohlc_data['date'] = datetime.datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')
# PROD
# ohlc_data['symbol'] = bitstamp_data['pair']

# TEST
ohlc_data['symbol'] = 'TEST'

# Convert the timestamp to datetime and localize it to UTC
dt = datetime.datetime.fromtimestamp(int(ohlc_data['unix']), tz=pytz.UTC)
# Convert the datetime to Kuala Lumpur time
kl_time = dt.astimezone(pytz.timezone('Asia/Kuala_Lumpur'))
ohlc_data['local_date'] = kl_time.strftime('%Y-%m-%dT%H:%M:%S')

# Convert 'volume' and 'close' to float before multiplication
ohlc_data['volume_usd'] = float(ohlc_data['volume_btc']) * float(ohlc_data['close'])
ohlc_json = json.dumps(ohlc_data)
# Convert the string to a dictionary
ohlc_data_dict = json.loads(ohlc_json)
display(ohlc_json)

C:\Users\Zhong_ming_Tan\AppData\Local\Temp\ipykernel_42892\2063910475.py:2: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  ohlc_data['date'] = datetime.datetime.utcfromtimestamp(int(ohlc_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')


'{"close": "51553", "high": "51763", "low": "51441", "open": "51735", "unix": "1708873200", "volume_btc": "42.58957250", "date": "2024-02-25T15:00:00", "symbol": "TEST", "local_date": "2024-02-25T23:00:00", "volume_usd": 2195620.2310925}'

In [19]:
ohlc_json

'{"close": "51553", "high": "51763", "low": "51441", "open": "51735", "unix": "1708873200", "volume_btc": "42.58957250", "date": "2024-02-25T15:00:00", "symbol": "TEST", "local_date": "2024-02-25T23:00:00", "volume_usd": 2195620.2310925}'

In [25]:
# Convert the string to a dictionary
ohlc_data_dict = json.loads(ohlc_json)

# Send the data to the API with a POST request
response = requests.post(hour_url, json=ohlc_data_dict)


In [24]:
response.text

'{"id":6760,"unix":1708873200,"date":"2024-02-25T15:00:00Z","local_date":"2024-02-25T23:00:00Z","symbol":"TEST","open":"51735.0000000000","high":"51763.0000000000","low":"51441.0000000000","close":"51553.0000000000","volume_btc":"42.5895725000","volume_usd":"2195620.2310925000","added_at":"2024-02-25T15:56:18.102090Z"}'

In [27]:
try:
    response = requests.post(hour_url, json=ohlc_data_dict)
    if response.status_code == 200:
        print('Data successfully added to the database')
    else:

        error_data = {
            "error_source": 'Hourly Bitstamp API',
            "error_type": response.status_code,
            "error_content": response.text,
            "error_message": "Calling bitstamp store api"
        }
        requests.post(error_log_url, json=error_data)
except requests.exceptions.RequestException as e:
    error_data = {
    "error_source": 'Hourly Bitstamp API',
    "error_type": e,
    "error_content": e.to_string(),
    "error_message": "Calling bitstamp store api"
    }
    requests.post(error_log_url, json=error_data)
    print(e)

Data successfully added to the database


=================


In [40]:
bitstamp_data['unix'] = bitstamp_data.pop('timestamp')
# Convert the Unix timestamp to UTC time
bitstamp_data['date'] = datetime.datetime.utcfromtimestamp(int(bitstamp_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')
bitstamp_data['symbol'] = "BTC/USD"

C:\Users\Zhong_ming_Tan\AppData\Local\Temp\ipykernel_18692\2030987883.py:3: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  bitstamp_data['date'] = datetime.datetime.utcfromtimestamp(int(bitstamp_data['unix'])).strftime('%Y-%m-%dT%H:%M:%S')


In [39]:
bitstamp_data_json = json.dumps(bitstamp_data)
print(bitstamp_data_json)

{"timestamp": "1708872619", "open": "51564", "high": "51870", "low": "51076", "last": "51738", "volume": "532.61435771", "vwap": "51574", "bid": "51731", "ask": "51737", "side": "0", "open_24": "51083", "percent_change_24": "1.28", "local_date": "2024-02-25T22:50:19"}


In [22]:
# Define the API endpoint
url = "http://127.0.0.1:8000/scrap/api/bitstamp/add"

In [18]:
# Send the data to the API with a POST request
response = requests.post(url, json=response.json())

Get Historical Data

In [12]:
# import pandas as pd
# data_json = data.json()
# df = pd.DataFrame(data_json, index=[0])
# df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Kuala_Lumpur')

In [50]:
import json
import requests
import pandas as pd
import datetime
from datetime import date, timedelta

currency_pair = "btcusd"
url = f"https://www.bitstamp.net/api/v2/ohlc/{currency_pair}/"

start = "2021-01-01"
end = "2021-01-02"

today = date.today()
# Get yesterday's date
yesterday = today - timedelta(days=1)
print("Today's date:", today.strftime('%Y-%m-%d'), "Yesterday's date:", yesterday.strftime('%Y-%m-%d'))

dates = pd.date_range(today.strftime('%Y-%m-%d'), today.strftime('%Y-%m-%d'), freq = "1H")
dates  = [ int(x.value/10**9) for x in list(dates)]

print(dates)

master_data = []

for first, last in zip(dates, dates[1:]):
    print(first,last)

    params = {
            "step":60,
            "limit":1,
            "start":first,
            "end":last
            }

    data = requests.get(url, params = params)

    data = data.json()["data"]["ohlc"]

    master_data += data

Today's date: 2024-02-25 Yesterday's date: 2024-02-24
[1708819200]


C:\Users\Zhong_ming_Tan\AppData\Local\Temp\ipykernel_20520\360508556.py:18: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  dates = pd.date_range(today.strftime('%Y-%m-%d'), today.strftime('%Y-%m-%d'), freq = "1H")


In [51]:
df = pd.DataFrame(master_data)
df.head()

""


In [47]:
import pandas as pd
df['local_date'] = pd.to_datetime(df['timestamp'], unit='s').dt.tz_localize('UTC').dt.tz_convert('Asia/Kuala_Lumpur')

KeyError: 'timestamp'

In [43]:
df.head(60)

,close,high,low,open,timestamp,volume,local_date
0,50858,50884,50855,50884,1708736400,0.06373954,2024-02-24 08:59:44+08:00
1,50622,50622,50592,50592,1708740000,0.01454434,2024-02-24 09:59:28+08:00
2,50764,50789,50759,50789,1708743600,0.02827627,2024-02-24 10:59:12+08:00
3,50839,50841,50834,50841,1708747200,0.58190849,2024-02-24 12:01:04+08:00
4,50975,50975,50946,50946,1708750800,0.01132726,2024-02-24 13:00:48+08:00
5,51038,51043,51038,51041,1708754400,0.03393921,2024-02-24 14:00:32+08:00
6,50954,50954,50944,50946,1708758000,0.01913675,2024-02-24 15:00:16+08:00
7,51064,51064,51061,51061,1708761600,0.00632000,2024-02-24 16:00:00+08:00
8,50984,50984,50984,50984,1708765200,0.00297000,2024-02-24 16:59:44+08:00
9,51092,51092,51092,51092,1708768800,0.17028794,2024-02-24 17:59:28+08:00
